In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#!pwd
%cd drive/My\ Drive
%cd Colab\ Notebooks
%cd Loan\ Project
# we are in the "Loan Project directory" - if there is an issue use "%cd ... to reenter directory"
#!ls
#!cat LoanStats3a_securev1_LendingClub_07-11.csv

/content/drive/My Drive
/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/Loan Project


In [5]:
df_07_11 = pd.read_csv('LoanStats3a_securev1_LendingClub_07-11.csv', sep=",", skiprows=1)
df_12_13 = pd.read_csv('LoanStats3b_securev1_LendingClub_12-13.csv', sep=",", skiprows=1)
df_14 = pd.read_csv("LoanStats3c_LendingClub_2014.csv", sep=",", skiprows=1)
df_15 = pd.read_csv('LoanStats3d_LendingClub_2015.csv', sep=",", skiprows=1)
df_16Q1 = pd.read_csv('LoanStats_LendingClub_2016Q1.csv', sep=",", skiprows=1)
df_16Q2 = pd.read_csv('LoanStats_LendingClub_2016Q2.csv', sep=",", skiprows=1)
df_16Q3 = pd.read_csv('LoanStats_LendingClub_2016Q3.csv', sep=",", skiprows=1)
df_16Q4 = pd.read_csv('LoanStats_LendingClub_2016Q4.csv', sep=",", skiprows=1)
df_17Q1 = pd.read_csv('LoanStats_LendingClub_2017Q1.csv', sep=",", skiprows=1)
df_17Q2 = pd.read_csv('LoanStats_LendingClub_2017Q2.csv', sep=",", skiprows=1)
df_17Q3 = pd.read_csv('LoanStats_LendingClub_2017Q3.csv', sep=",", skiprows=1)
df_17Q4 = pd.read_csv('LoanStats_LendingClub_2017Q4.csv', sep=",", skiprows=1)
df_18Q1 = pd.read_csv('LoanStats_LendingClub_2018Q1.csv', sep=",", skiprows=1)
df_18Q2 = pd.read_csv('LoanStats_LendingClub_2018Q2.csv', sep=",", skiprows=1)
df_18Q3 = pd.read_csv('LoanStats_LendingClub_2018Q3.csv', sep=",", skiprows=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, 

In [0]:
frames = [df_07_11, df_12_13, df_14, df_15, df_16Q1, df_16Q2, df_16Q3, df_16Q4, df_17Q1, df_17Q2, df_17Q3, df_17Q4, df_18Q1,df_18Q2,df_18Q3]

result = pd.concat(frames, ignore_index=True)

In [8]:
result.shape

(2132287, 151)

In [11]:
result.drop(columns=['id', 'member_id','url','zip_code'], inplace=True)
result.shape

(2132287, 147)

In [12]:
#"loan_status" is the column heading of whether it defulted or not
x = result.copy()
#labels = df.loc[:,"loan_status"]
y = result['loan_status'].copy()
y.shape[0]


2132287

In [15]:
label = []
for i in range(y.shape[0]):
  if y[i] not in label:
    label.append(y[i])

for i in range(len(label)):
  print(label[i])
  
print("There are", len(label), "labels")

Fully Paid
Charged Off
nan
Does not meet the credit policy. Status:Fully Paid
Does not meet the credit policy. Status:Charged Off
Late (31-120 days)
Current
Late (16-30 days)
In Grace Period
Default
There are 10 labels


##NOTES about loan_status (aka. labels):


*   if label is "nan" - we throw that loan out of the training set
*   Current means that the loan is still being paid off, and therefore we cannot use for out training - might be useful to comeback and revisit in the future (maybe predict outcome now) - will be thrown out of training set
*   fully paid  - successful loan


---


*   what is "charged off" vs "default"
 *   "Loans that are in "Default" are loans for which borrowers have failed to make payments for an extended period of time. A loan becomes “Charged Off” when there is no longer a reasonable expectation of further payments.  Charge Off typically occurs when a loan is 120 days or more past due and there is no reasonable expectation of sufficient payment to prevent the charge off.  In certain circumstances, loans may be charged off at an earlier or later date.  Please note, loans for which borrowers have filed for bankruptcy may be charged off earlier based on the date of bankruptcy notification.
 A loan that is in “Default” will still appear in your Notes, in the status of “Default,” while a loan that has been “Charged Off” will appear as charged off, and the remaining principal balance of the Note will be deducted from your account balance.  Learn more about what happens when a loan is charged off"
 *   Because in the case of "charged off" vs "default" the lender has extremely low chance of recovering their investment, we will treat both of these classifications as if they are the same thing

---


*   what does it mean if it "does not meet the credit policy"
 *   Lending Club doesn't make clear the meaning of this statement, but from general internet research most ppl just ommit the "does not meet the credit policy" part and use the second half of the classification
---
*   what does "grace period" mean - should we potentially throw those away?
 * Loan is past due but within the 15-day grace period. - we will throw these away because for training data it isn't useful since the final status of loan isn't avaliable
*   if its late - does that mean status is still being finalized? - should we throw away?
 * if it is late we don't use this loan in training set cus finalized status isn't avalible to confirm predictions

